# Dim Turno

In [0]:
import pyspark.sql.functions as f

In [0]:
spark.sql('USE CATALOG saude_sus')

In [0]:
df_tru = spark.read.table('saude_sus.trusted.tru_estabelecimento')

In [0]:
from pyspark.sql import Window

window_spec = Window.partitionBy("COD_IDENTIFICADOR_TURNO_ATENDIMENTO").orderBy(f.col("DAT_ATUALIZACAO").desc())

df_dim_turno = (
    df_tru
    .where(f.col("COD_IDENTIFICADOR_TURNO_ATENDIMENTO").isNotNull())
    .withColumn("row_num", f.row_number().over(window_spec))
    .filter("row_num = 1")
    .select(
        f.md5(f.col("COD_IDENTIFICADOR_TURNO_ATENDIMENTO").cast("string")).alias("SK_TURNO"),
        f.col("COD_IDENTIFICADOR_TURNO_ATENDIMENTO"),
        f.col("DSC_TURNO_ATENDIMENTO")
    )
)

row_unknown = spark.createDataFrame([("-1", "00", "NÃO INFORMADO")], 
                                   ["SK_TURNO", "COD_IDENTIFICADOR_TURNO_ATENDIMENTO", "DSC_TURNO_ATENDIMENTO"])

df_dim_turno_final = df_dim_turno.union(row_unknown)

df_dim_turno_final.write.mode("overwrite").saveAsTable("saude_sus.refined.dim_turno")